In [304]:

import pandas as pd
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
import keras


import fix_yahoo_finance as fyf
from datetime import datetime
from pandas.tseries.offsets import BDay
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

In [357]:
df = pd.read_csv("C:/Users/patri/Desktop/Python/AtomPython/SEB/Data/News.csv")

När vi sedan läst in datan är det dags att konvertera samtliga ord för varje nyhetet till en integer.
Detta görs först igenom att skapa en dict där "Key" motsvara samtliga ord som använts i vårat dataset.
Varje "Value" motsvarar en integer.
dictionary:n benämns "words". 

In [358]:
news = df["News"].tolist()

In [359]:
words = {}
counter = 0
for i in range(len(news)):
    txt = news[i].split(" ")
    for p in txt:
        wrd = p.lower()
        wrd = wrd.replace("." , "")
        wrd = wrd.replace("," , "")
        if wrd in list(words.keys()):
            continue
        else:
            words[wrd] = counter
        counter += 1
    if i % 100 ==0:
        print(i)


0
100
200
300
400
500


Vi skapar sedan "list" för varje interimsrapport med en korresponderande integer för varje ord
enligt den "dictionary" vi skapade tidigare.


In [360]:
news_index =  []
for i in range(len(news)):
    txt = news[i].split(" ")
    encode_text = []
    for p in txt:
        wrd = p.lower()
        wrd = wrd.replace("." , "")
        wrd = wrd.replace("," , "")
        encode_text.append(words[wrd]) 
    news_index.append(encode_text)
    
        

För att kontrollera vi har indexerad samtliga ord korrekt byter vi sedan plats på var "key-value pair" i "words" och benäner näst dictionary index_word
Sedan kan vi jämför två texter för att se att detta stämmer

In [361]:
index_word = {v:k for k, v in words.items()}

In [368]:
message = ""
for i in news_index[10]:
    message = message + f"{index_word[i]} "
print(message)
print("\n")
print("=" * 125)
print("\n")
print(news[10])

third quarter 1 october – 31 december 2018  · net revenue increased by 14 percent to msek 1011 (888) organically netrevenue increased by 7 percent  · operating profit (ebita) increased by 23 percent to msek 137 (111)equivalent to an operating margin of 136 percent (125)  · profit after financial items increased by 25 percent to msek 114 (91)  · profit after taxes increased by 26 percent to msek 88 (70) earnings pershare for the latest 12-month period increased by 18 percent to sek 482 (407)  · return on equity was 24 percent (23) the equity ratio at the end of theperiod was 39 percent (36)  · cash flow from operating activities for the latest 12-month period amountedto msek 430 compared to msek 282 for the 2017/18 financial year  · during the quarter a purchase agreement was signed to acquire just over 90percent of the shares in schmitztechnik gmbh with annual revenue of about meur67 the shares were taken into possession on 8 january 2019the first nine months (1 april – 31 december 201

En kort koll säger att vi har ca 270 bolag att arbeta med.
Problemet är att det inte går att få korrekta historiska aktieprisdata med de bolags namn vi har nedan.

Lösningen blir att hämta dessa från Yahoo Finance vilket det finns bra bibliotek för.
Problemet här är bara man måste ange Ticker för att kunna hämta aktuell aktie och alla kommer inte att finnas på yahoo. 

För varje bolag tvingas jag då gå upp på yahoo manuellt och hämta korrekt ticker i en loop enligt nedan. 

In [52]:
df["Company"].unique()

array(['Arion banki hf.', 'Recipharm AB', 'Adapteo Oyj',
       'Royal Bank of Canada', 'Castellum AB', 'SAAB AB', 'Edgeware AB',
       'AddLife AB', 'Swedbank AB', 'Lagercrantz Group AB',
       'Swedbank Hypotek AB', 'Addtech AB', 'Brado AB', 'Elekta AB',
       'Lauritz.com A/S', 'Klarna Bank AB (publ)', 'Strax AB',
       'Karnov Group AB', 'Medivir AB', 'Sunborn (Gibraltar) Limited',
       'Ferronordic AB',
       'Internationella Engelska Skolan i Sverige Holdings II AB',
       'NeuroVive Pharmaceutical AB', 'Gambling.com Group Plc',
       'Zengun Group AB', 'Catena Media P.L.C', 'Oncopeptides AB',
       'Moberg Pharma AB', 'SSM Holding AB', 'Arctic Paper AB',
       'Baltic Horizon Fund / Northern Horizon Capital',
       'European Directories BondCo S.C.A', 'Cantargia AB',
       'Etrion Corporation', 'Bong AB', 'Karo Pharma AB', 'Björn Borg AB',
       'Bayport Management LTD', 'Jetpak Top Holding AB',
       'RaySearch Laboratories AB', 'Opus Group AB', 'Catella AB',
   

In [8]:
companies = df["Company"].unique().tolist()
tickers = {}
for company in companies:
    print(company)
    ticker = input("Company Ticker")
    if ticker.lower() != "n" :
        if company not in list(tickers.keys()):
            tickers[company] = [ticker, 2019, 1]
        else:
            pass

Arion banki hf.
Company TickerARION-SDB.ST
Recipharm AB
Company TickerRECI-B.ST
Adapteo Oyj
Company TickerADAPT.ST
Royal Bank of Canada
Company Tickern
Castellum AB
Company TickerCAST.ST
SAAB AB
Company TickerSAAB-B.ST
Edgeware AB
Company TickerEDGE.ST
AddLife AB
Company TickerALIF-B.ST
Swedbank AB
Company TickerSWED-A.ST
Lagercrantz Group AB
Company TickerLAGR-B.ST
Swedbank Hypotek AB
Company Tickern
Addtech AB
Company TickerADDT-B.ST
Brado AB
Company Tickern
Elekta AB
Company TickerEKTA-B.ST
Lauritz.com A/S
Company TickerLAUR.ST
Klarna Bank AB (publ)
Company Tickern
Strax AB
Company TickerSTRAX.ST
Karnov Group AB
Company TickerKAR.ST
Medivir AB
Company TickerMVIR-B.ST
Sunborn (Gibraltar) Limited
Company Tickern
Ferronordic AB
Company TickerFNM.ST
Internationella Engelska Skolan i Sverige Holdings II AB
Company TickerENG.ST
NeuroVive Pharmaceutical AB
Company TickerNVP.ST
Gambling.com Group Plc
Company Tickern
Zengun Group AB
Company Tickern
Catena Media P.L.C
Company TickerCTM.ST
Onc

Company TickerSPOR.ST
Svensk Exportkredit AB
Company Tickern
Atlas Copco AB
Company TickerATCO-A.ST
Byggmax Group AB
Company TickerBMAX.ST
Fabege AB
Company TickerFABG.ST
Sandvik AB
Company TickerSAND.ST
OEM International AB
Company TickerOEM-B.ST
Investor AB
Company TickerINVE-B.ST
Getinge AB
Company TickerGETI-B.ST
ASSA ABLOY AB
Company TickerASSA-B.ST
DUNI AB
Company TickerDUNI.ST
Volvo, AB
Company TickerVOLV-B.ST
Vitec Software Group AB
Company TickerVIT-B.ST
Nederman Holding AB
Company TickerNMAN.ST
Ericsson, Telefonab. L M
Company TickerERIC-B.ST
Telia Company AB
Company TickerTELIA.ST
Tele2 AB
Company TickerTEL2-B.ST
Hennes & Mauritz AB, H & M
Company TickerHM-B.ST
Nobina AB
Company TickerNOBINA.ST
Morgan Stanley B.V
Company Tickern
DSV Miljø Group A/S
Company Tickern
Cipp Technology Solutions A/S
Company Tickern
Clas Ohlson AB
Company TickerCLAS-B.ST
SECTRA AB
Company TickerSECT-B.ST
SB Holdco PLC
Company Tickern
Real People Investment Holdings Limited
Company Tickern
Global Sc

När detta är klart kan vi definera de funktioner jag behöver för att hämta datan.
"Get_data" går upp och hämtar aktuell ticker och kräver att man anger ett start och slut datum. 

In [15]:
def get_data(symbol, start_date, end_date):
    panel_data = fyf.download(symbol, start_date, end_date)
    
    panel_data["ratio"] = panel_data["Adj Close"] / panel_data["Close"]
    
    panel_data["Close"] =  panel_data["Close"] * panel_data["ratio"]
    panel_data["Open"] =  panel_data["Open"] * panel_data["ratio"]
    panel_data["High"] =  panel_data["High"] * panel_data["ratio"]
    panel_data["Low"] =  panel_data["Low"] * panel_data["ratio"]
    

    return panel_data

Vi ser till att spara ned samtliga tickers i csv format ifall vi göra en hämtning i framtiden. 

In [10]:
cp = list(tickers.keys())
tic = []

for i in cp:
    tic.append(tickers[i][0])

In [11]:
df1 = pd.DataFrame({"Company Name": cp, 
                   "Yahoo Ticker": tic})
df1.to_csv("company-ticker.csv", index=False)

En enkel For-loop för att iterera över samtliga bolag och ta ned korrekt aktieprisdata.

In [20]:
end =datetime(2019, 12, 31) 
for ticker in tickers.keys():
    try:
        start = datetime(tickers[ticker][1], 1, tickers[ticker][2]) 
        df = get_data(tickers[ticker][0], start, end)
        name = f"C:/Users/patri/Desktop/Python/AtomPython/SEB/Data/Stockprice/{tickers[ticker][0]}.csv"
        df.to_csv(name)
    except ValueError:
        print("fel av " + ticker)        


[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*******************

I förra loopen glömde jag att låta bli att benämnda "DAtaframe" till något annat än "df" som våran orginaldata redan är benämn. 
Får läsa in datan på nytt.
Lägger även till våran processade data "News_index"
Konverterar "Tidpunkten" till datetime enligt pandas
Filtrerar bort samtliga bolag som vi antingen inte fanns på  Yahoo eller inte gick att ladda ned. 
Skapar Datum och års kolumner 
Filtrera bort alla nyheter från 2020.

In [330]:
df = pd.read_csv("C:/Users/patri/Desktop/Python/AtomPython/SEB/Data/News.csv")
df["News Index"] = news_index
df["TimeStamp"] = pd.to_datetime(df["TimeStamp"])
df = df[df['Company'].isin(cp)]
df = df[df['Company'] != "Camurus AB"]
df = df[df['Company'] != "Midsona AB"]
df = df[df['Company'] != "Leax Group AB"]
df["Date"] = df["TimeStamp"].dt.date
df["Year"] = df["TimeStamp"].dt.year

df = df[df['Year'] != 2020]



Näst mål är att få fram hur aktiens pris förändrades i samband med interimsrapporten.
Vi skapar "return_func" som skall gå igenom datasetet och ta fram vad aktien stängde på den senaste "Business Day" innan rapporten släpptes.
Även ta fram vad aktien stänger på nästa "Business Day" efter rapporten släpptes.
Få fram vad förändringen är 
Sedan skapa tre klasser:
"Över Tre procenst uppsida"
"Under Tre procents nedsida"
"Plus/minus 3 procent"




In [331]:
def return_func(dataframe):
    cmp = dataframe["Company"]
    time = dataframe["Date"]
    tick = tickers[cmp][0]
    
    day_before = time - BDay(1)
    day_after = time + BDay(1)


    
    try:
        read  = pd.read_csv(f"C:/Users/patri/Desktop/Python/AtomPython/SEB/Data/Stockprice/{tick}.csv")
        read["Date"] = pd.to_datetime(read["Date"])
    except:
        print(cmp)
    
    if read.shape[0] > 246:
        temp_df = read.loc[read["Date"] == day_before]
        while temp_df.shape[0] == 0:
            day_before = day_before - BDay(1)
            temp_df = read.loc[read["Date"] == day_before]
            
        day_before_closing = temp_df.iloc[0]["Close"] * temp_df.iloc[0]["ratio"]
        
        temp_df = read.loc[read["Date"] == day_after]
        
        while temp_df.shape[0] == 0:
            day_after = day_after + BDay(1)
            temp_df = read.loc[read["Date"] == day_after]
            
        day_after_closing = temp_df.iloc[0]["Close"] * temp_df.iloc[0]["ratio"]
        
        return_value = (day_after_closing / day_before_closing)-1
        
    else:
        
        try:
            temp_df = read.loc[read["Date"] == day_before]
            if temp_df.shape[0] == 0:
                day_before = day_before - BDay(1)
                temp_df = read.loc[read["Date"] == day_before]

            day_before_closing = temp_df.iloc[0]["Close"] * temp_df.iloc[0]["ratio"]  
        

            temp_df = read.loc[read["Date"] == day_after]

            if temp_df.shape[0] == 0:
                day_after = day_after + BDay(1)
                temp_df = read.loc[read["Date"] == day_after]

            day_after_closing = temp_df.iloc[0]["Close"] * temp_df.iloc[0]["ratio"]
            
            return_value = (day_after_closing / day_before_closing)-1
            
        except:
            
            return_value="No-Data"
     
    dataframe["PercentChange"] = return_value
    
    if return_value != "No-Data":
        if return_value  > 0.03:
            classes = "Over 3"
        elif return_value < -0.03:
            classes = "Under 3"
        else:
            classes = "Minor Movement"
    else:
        classes = "No-Data"
        
    dataframe["Over/Under"] = classes
    
    return dataframe
            
            
        



In [332]:
df = df.apply(return_func, axis=1)

Nu tar vi bort alla aktier som inte handlats fler än 246 under 2019 dvs de som blivit nyintroducerade till Börsen.
Vi sätter in classerna i integers istället för "Over 3" "under 3" och "Minor Movement"

In [333]:
df = df[df["Over/Under"] != "No-Data"]
encoder = LabelEncoder()
df["Classes"] = encoder.fit_transform(df["Over/Under"].values)

Äntligen närmar vi oss själva Modellen. 

Vi skapar x, y samt splittar dessa i traingset och testset.

Vi använder oss av Keras Preprocessing och framförallt Tokenizer.
Tokenizer konverterar datan till representerbar data för det neurala nätverket. 

vi använder endast de 10 000 mest förekommande orden i våran input.

Vi kommer test tre olika  "modes"

Binary - gör helt "Dummy Variables" av text-datan.
Count - Återskapar hur många gånger ett ord förekommer i texten.
Frequency - Återskapar hur freqvent ett ord används i förhållande till andra ord.

In [335]:
x = np.array(df["News Index"])
y = np.array(df["Classes"])

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

tokenizer  = Tokenizer(num_words = 10000)


x_train = tokenizer.sequences_to_matrix(x_train, mode="binary")
x_test = tokenizer.sequences_to_matrix(x_test, mode="binary")

y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)




modellen.  
Keras Sequential med två "Dense Layers" samt dropout mellan samtliga. Vi kommer se att modellen overfittar väldigt snabbt.
Optimizer är "adam" mest för att den är vanligast förekommande.
Learning Rate har sänkts ordentligt för att får en långsammare "overfitting" och komma närmre traingsdata med testdatan i performance.

Vi använder en early stopping för att undvika så mycket overfitting som möjligt.

In [337]:
model = Sequential()
model.add(Dense(512, input_dim=x_train.shape[1], activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation="softmax"))
optimizer = keras.optimizers.adam(lr=0.0001)
early_stopper = keras.callbacks.EarlyStopping(monitor="val_acc", patience=20, restore_best_weights=True)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(x_train, y_train, validation_data=[x_test, y_test], epochs=1000, batch_size=32, callbacks=[early_stopper])


Train on 352 samples, validate on 89 samples
Epoch 1/1000
352/352 [==============================] - 2s 5ms/step - loss: 1.1045 - acc: 0.3608 - val_loss: 1.0849 - val_acc: 0.3258
Epoch 2/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.0342 - acc: 0.4631 - val_loss: 1.0641 - val_acc: 0.3820
Epoch 3/1000
352/352 [==============================] - 1s 4ms/step - loss: 0.9787 - acc: 0.5369 - val_loss: 1.0592 - val_acc: 0.4045
Epoch 4/1000
352/352 [==============================] - 1s 4ms/step - loss: 0.9308 - acc: 0.6250 - val_loss: 1.0640 - val_acc: 0.4157
Epoch 5/1000
352/352 [==============================] - 1s 4ms/step - loss: 0.8527 - acc: 0.7216 - val_loss: 1.0703 - val_acc: 0.3708
Epoch 6/1000
352/352 [==============================] - 1s 4ms/step - loss: 0.8023 - acc: 0.7443 - val_loss: 1.0740 - val_acc: 0.4045
Epoch 7/1000
352/352 [==============================] - 1s 4ms/step - loss: 0.7436 - acc: 0.7955 - val_loss: 1.0755 - val_acc: 0.4157
Epoch 8/1000
352/

In [338]:
pred = model.predict(x_test)
pred = [np.argmax(pred[i]) for i in range(len(pred))]
validation =[np.argmax(y_test[i]) for i in range(len(y_test))]


print(accuracy_score(pred, validation))
print(classification_report(pred, validation))

0.43820224719101125
              precision    recall  f1-score   support

           0       0.53      0.49      0.51        39
           1       0.50      0.42      0.46        43
           2       0.12      0.29      0.17         7

    accuracy                           0.44        89
   macro avg       0.38      0.40      0.38        89
weighted avg       0.48      0.44      0.46        89



In [339]:
keras.backend.clear_session()

In [340]:
x = np.array(df["News Index"])
y = np.array(df["Classes"])

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

tokenizer  = Tokenizer(num_words = 10000)


x_train = tokenizer.sequences_to_matrix(x_train, mode="count")
x_test = tokenizer.sequences_to_matrix(x_test, mode="count")

y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)




In [341]:
model = Sequential()
model.add(Dense(512, input_dim=x_train.shape[1], activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation="softmax"))
optimizer = keras.optimizers.adam(lr=0.0001)
early_stopper = keras.callbacks.EarlyStopping(monitor="val_acc", patience=20, restore_best_weights=True)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(x_train, y_train, validation_data=[x_test, y_test], epochs=1000, batch_size=32, callbacks=[early_stopper])


Train on 352 samples, validate on 89 samples
Epoch 1/1000
352/352 [==============================] - 2s 5ms/step - loss: 1.8693 - acc: 0.3835 - val_loss: 1.4990 - val_acc: 0.4382
Epoch 2/1000
352/352 [==============================] - 1s 4ms/step - loss: 2.2248 - acc: 0.4176 - val_loss: 1.8209 - val_acc: 0.4494
Epoch 3/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.4939 - acc: 0.4517 - val_loss: 1.7909 - val_acc: 0.3933
Epoch 4/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.6054 - acc: 0.4545 - val_loss: 1.3371 - val_acc: 0.4494
Epoch 5/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.5415 - acc: 0.5284 - val_loss: 1.2801 - val_acc: 0.4607
Epoch 6/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.3553 - acc: 0.5398 - val_loss: 1.3072 - val_acc: 0.4494
Epoch 7/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.5546 - acc: 0.5597 - val_loss: 1.3426 - val_acc: 0.4607
Epoch 8/1000
352/

In [347]:
pred = model.predict(x_test)
pred = [np.argmax(pred[i]) for i in range(len(pred))]
validation =[np.argmax(y_test[i]) for i in range(len(y_test))]


print(accuracy_score(pred, validation))
print(classification_report(pred, validation))

0.449438202247191
              precision    recall  f1-score   support

           0       0.89      0.42      0.57        76
           1       0.22      0.62      0.33        13
           2       0.00      0.00      0.00         0

    accuracy                           0.45        89
   macro avg       0.37      0.35      0.30        89
weighted avg       0.79      0.45      0.54        89



In [348]:
keras.backend.clear_session()

In [349]:
x = np.array(df["News Index"])
y = np.array(df["Classes"])

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

tokenizer  = Tokenizer(num_words = 10000)


x_train = tokenizer.sequences_to_matrix(x_train, mode="freq")
x_test = tokenizer.sequences_to_matrix(x_test, mode="freq")

y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)




In [350]:
model = Sequential()
model.add(Dense(512, input_dim=x_train.shape[1], activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation="softmax"))
optimizer = keras.optimizers.adam(lr=0.0001)
early_stopper = keras.callbacks.EarlyStopping(monitor="val_acc", patience=20, restore_best_weights=True)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(x_train, y_train, validation_data=[x_test, y_test], epochs=1000, batch_size=32, callbacks=[early_stopper])


Train on 352 samples, validate on 89 samples
Epoch 1/1000
352/352 [==============================] - 2s 5ms/step - loss: 1.0979 - acc: 0.4119 - val_loss: 1.0964 - val_acc: 0.3933
Epoch 2/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.0968 - acc: 0.4119 - val_loss: 1.0943 - val_acc: 0.4157
Epoch 3/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.0948 - acc: 0.4091 - val_loss: 1.0922 - val_acc: 0.4045
Epoch 4/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.0937 - acc: 0.4205 - val_loss: 1.0900 - val_acc: 0.4157
Epoch 5/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.0918 - acc: 0.4403 - val_loss: 1.0876 - val_acc: 0.4157
Epoch 6/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.0900 - acc: 0.4347 - val_loss: 1.0850 - val_acc: 0.4157
Epoch 7/1000
352/352 [==============================] - 1s 4ms/step - loss: 1.0885 - acc: 0.4261 - val_loss: 1.0823 - val_acc: 0.4045
Epoch 8/1000
352/

In [351]:
pred = model.predict(x_test)
pred = [np.argmax(pred[i]) for i in range(len(pred))]
validation =[np.argmax(y_test[i]) for i in range(len(y_test))]


print(accuracy_score(pred, validation))
print(classification_report(pred, validation))

0.4943820224719101
              precision    recall  f1-score   support

           0       0.61      0.52      0.56        42
           1       0.56      0.48      0.51        42
           2       0.12      0.40      0.18         5

    accuracy                           0.49        89
   macro avg       0.43      0.47      0.42        89
weighted avg       0.56      0.49      0.52        89



Som Vi ser har modellen svårt att predicta news.
Det beror främst på att datasetet är väldigt litet. Modellen får heller inte någon input på hur aktien har preseterat exempelvis
de senaste 30 dagarna vilket kan ha står påverkan på om rapporten är "bra " eller "Dålig".

Dock superintressant att den sätter nästan hälften av rörelsena baserat på själva rapporten. 